In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template
load_figure_template("sketchy")
#from plotly.io import templates

In [3]:
df=pd.read_csv('data/complete_dataset.csv')

In [4]:
df.rename(columns={'year': 'year_emission'}, inplace=True)
df.rename(columns={'country': 'country_happy'}, inplace=True)
df.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)

In [5]:
# calculate gdp per capita
df['gdp_capita'] = df['gross_domestic_product_(gdp)'] / df['population']

# calculate the percentages for mining, manufacturing, utilities and other activities regarding the total value added
df['p_agriculture,_hunting,_forestry,_fishing'] = (df['agriculture,_hunting,_forestry,_fishing_(isic_a-b)'] / df['gross_domestic_product_(gdp)']) * 100
df['p_mining_manufacturing_utilities'] = (df['mining,_manufacturing,_utilities_(isic_c-e)'] / df['gross_domestic_product_(gdp)']) * 100
df['p_other_activities'] = (df['other_activities_(isic_j-p)'] / df['gross_domestic_product_(gdp)']) * 100
df['p_wholesale_trade_restaurants'] = (df['wholesale,_retail_trade,_restaurants_and_hotels_(isic_g-h)'] / df['gross_domestic_product_(gdp)']) * 100
df['p_transport,_storage_and_communication_(isic_i)'] = (df['transport,_storage_and_communication_(isic_i)'] / df['gross_domestic_product_(gdp)']) * 100
df['p_general_government_final_consumption_expenditure'] = (df['general_government_final_consumption_expenditure'] / df['gross_domestic_product_(gdp)']) * 100
df['p_household_consumption_expenditure']= (df['household_consumption_expenditure_(including_non-profit_institu'] / df['gross_domestic_product_(gdp)']) * 100

# calculate the sum of export and import
df['sum_export_import'] = df['exports_of_goods_and_services'] - df['imports_of_goods_and_services'] 



In [6]:
# cut off years after 2018
df = df[df['year'] <= 2018]

In [7]:
# drop some of the unneeded columns

columns_to_drop = ['ama_exchange_rate', 'country_name', 'countryid', 'currency', 'changes_in_inventories', 'gross_capital_formation',
                   'gross_fixed_capital_formation_(including_acquisitions_less_disp', 'manufacturing_(isic_d)',
                   'rank', 'year_emission', 'gdp_per_capita', 'country_happy', 'year_happy', 'country_le',
                    'code_le', 'year_le', 'country_dens', 'year_dens']
df = df.drop(columns=columns_to_drop)

In [8]:
# set all 0 values in emission to missing

df['emission'] = df['emission'].replace(0, pd.NA)
df['emission'] = pd.to_numeric(df['emission'], errors='coerce')

# calculate total emissions
df['emission_total'] = df['emission'] * df['population']
df['emission_total'] = pd.to_numeric(df['emission_total'], errors='coerce')


In [9]:
# calculate a value for emissions in regard to gdp
df['emission_gdp'] = (df['emission_total']/df['gross_domestic_product_(gdp)'])*100
df['emission_gdp'] = pd.to_numeric(df['emission_gdp'], errors='coerce')


In [10]:
# get the continent to all countries

country_to_continent = {
    'Afghanistan': 'Asia',
    'Albania': 'Europe',
    'Algeria': 'Africa',
    'Andorra': 'Europe',
    'Angola': 'Africa',
    'Anguilla': 'North America',
    'Antigua and Barbuda': 'North America',
    'Argentina': 'South America',
    'Armenia': 'Asia',
    'Aruba': 'North America',
    'Australia': 'Oceania',
    'Austria': 'Europe',
    'Azerbaijan': 'Asia',
    'Bahamas': 'North America',
    'Bahrain': 'Asia',
    'Bangladesh': 'Asia',
    'Barbados': 'North America',
    'Belarus': 'Europe',
    'Belgium': 'Europe',
    'Belize': 'North America',
    'Benin': 'Africa',
    'Bermuda': 'North America',
    'Bhutan': 'Asia',
    'Bolivia (Plurinational State of)': 'South America',
    'Bosnia and Herzegovina': 'Europe',
    'Botswana': 'Africa',
    'Brazil': 'South America',
    'British Virgin Islands': 'North America',
    'Brunei Darussalam': 'Asia',
    'Bulgaria': 'Europe',
    'Burkina Faso': 'Africa',
    'Burundi': 'Africa',
    'Cabo Verde': 'Africa',
    'Cambodia': 'Asia',
    'Cameroon': 'Africa',
    'Canada': 'North America',
    'Cayman Islands': 'North America',
    'Central African Republic': 'Africa',
    'Chad': 'Africa',
    'Chile': 'South America',
    'China': 'Asia',
    'China, Hong Kong SAR': 'Asia',
    'China, Macao SAR': 'Asia',
    'Colombia': 'South America',
    'Comoros': 'Africa',
    'Congo': 'Africa',
    'Cook Islands': 'Oceania',
    'Costa Rica': 'North America',
    'Croatia': 'Europe',
    'Cuba': 'North America',
    'Curaçao': 'North America',
    'Cyprus': 'Asia',
    'Czechia': 'Europe',
    'Czechoslovakia (Former)': 'Europe',
    "Côte d'Ivoire": 'Africa',
    'D.P.R. of Korea': 'Asia',
    'D.R. of the Congo': 'Africa',
    'Denmark': 'Europe',
    'Djibouti': 'Africa',
    'Dominica': 'North America',
    'Dominican Republic': 'North America',
    'Ecuador': 'South America',
    'Egypt': 'Africa',
    'El Salvador': 'North America',
    'Equatorial Guinea': 'Africa',
    'Eritrea': 'Africa',
    'Estonia': 'Europe',
    'Eswatini': 'Africa',
    'Ethiopia': 'Africa',
    'Ethiopia (Former)': 'Africa',
    'Fiji': 'Oceania',
    'Finland': 'Europe',
    'Former Netherlands Antilles': 'North America',
    'France': 'Europe',
    'French Polynesia': 'Oceania',
    'Gabon': 'Africa',
    'Gambia': 'Africa',
    'Georgia': 'Asia',
    'Germany': 'Europe',
    'Ghana': 'Africa',
    'Greece': 'Europe',
    'Greenland': 'North America',
    'Grenada': 'North America',
    'Guatemala': 'North America',
    'Guinea': 'Africa',
    'Guinea-Bissau': 'Africa',
    'Guyana': 'South America',
    'Haiti': 'North America',
    'Honduras': 'North America',
    'Hungary': 'Europe',
    'Iceland': 'Europe',
    'India': 'Asia',
    'Indonesia': 'Asia',
    'Iran (Islamic Republic of)': 'Asia',
    'Iraq': 'Asia',
    'Ireland': 'Europe',
    'Israel': 'Asia',
    'Italy': 'Europe',
    'Jamaica': 'North America',
    'Japan': 'Asia',
    'Jordan': 'Asia',
    'Kazakhstan': 'Asia',
    'Kenya': 'Africa',
    'Kiribati': 'Oceania',
    'Kosovo': 'Europe',
    'Kuwait': 'Asia',
    'Kyrgyzstan': 'Asia',
    "Lao People's DR": 'Asia',
    'Latvia': 'Europe',
    'Lebanon': 'Asia',
    'Lesotho': 'Africa',
    'Liberia': 'Africa',
    'Libya': 'Africa',
    'Liechtenstein': 'Europe',
    'Lithuania': 'Europe',
    'Luxembourg': 'Europe',
    'Madagascar': 'Africa',
    'Malawi': 'Africa',
    'Malaysia': 'Asia',
    'Maldives': 'Asia',
    'Mali': 'Africa',
    'Malta': 'Europe',
    'Marshall Islands': 'Oceania',
    'Mauritania': 'Africa',
    'Mauritius': 'Africa',
    'Mexico': 'North America',
    'Micronesia (FS of)': 'Oceania',
    'Monaco': 'Europe',
    'Mongolia': 'Asia',
    'Montenegro': 'Europe',
    'Montserrat': 'North America',
    'Morocco': 'Africa',
    'Mozambique': 'Africa',
    'Myanmar': 'Asia',
    'Namibia': 'Africa',
    'Nauru': 'Oceania',
    'Nepal': 'Asia',
    'Netherlands': 'Europe',
    'New Caledonia': 'Oceania',
    'New Zealand': 'Oceania',
    'Nicaragua': 'North America',
    'Niger': 'Africa',
    'Nigeria': 'Africa',
    'North Macedonia': 'Europe',
    'Norway': 'Europe',
    'Oman': 'Asia',
    'Pakistan': 'Asia',
    'Palau': 'Oceania',
    'Panama': 'North America',
    'Papua New Guinea': 'Oceania',
    'Paraguay': 'South America',
    'Peru': 'South America',
    'Philippines': 'Asia',
    'Poland': 'Europe',
    'Portugal': 'Europe',
    'Puerto Rico': 'North America',
    'Qatar': 'Asia',
    'Republic of Korea': 'Asia',
    'Republic of Moldova': 'Europe',
    'Romania': 'Europe',
    'Russian Federation': 'Europe',
    'Rwanda': 'Africa',
    'Saint Kitts and Nevis': 'North America',
    'Saint Lucia': 'North America',
    'Samoa': 'Oceania',
    'San Marino': 'Europe',
    'Sao Tome and Principe': 'Africa',
    'Saudi Arabia': 'Asia',
    'Senegal': 'Africa',
    'Serbia': 'Europe',
    'Seychelles': 'Africa',
    'Sierra Leone': 'Africa',
    'Singapore': 'Asia',
    'Sint Maarten (Dutch part)': 'North America',
    'Slovakia': 'Europe',
    'Slovenia': 'Europe',
    'Solomon Islands': 'Oceania',
    'Somalia': 'Africa',
    'South Africa': 'Africa',
    'South Sudan': 'Africa',
    'Spain': 'Europe',
    'Sri Lanka': 'Asia',
    'St. Vincent and the Grenadines': 'North America',
    'State of Palestine': 'Asia',
    'Sudan': 'Africa',
    'Sudan (Former)': 'Africa',
    'Suriname': 'South America',
    'Sweden': 'Europe',
    'Switzerland': 'Europe',
    'Syrian Arab Republic': 'Asia',
    'Tajikistan': 'Asia',
    'Thailand': 'Asia',
    'Timor-Leste': 'Asia',
    'Togo': 'Africa',
    'Tonga': 'Oceania',
    'Trinidad and Tobago': 'North America',
    'Tunisia': 'Africa',
    'Turkmenistan': 'Asia',
    'Turks and Caicos Islands': 'North America',
    'Tuvalu': 'Oceania',
    'Türkiye': 'Europe',
    'U.R. of Tanzania: Mainland': 'Africa',
    'USSR (Former)': 'Europe',
    'Uganda': 'Africa',
    'Ukraine': 'Europe',
    'United Arab Emirates': 'Asia',
    'United Kingdom': 'Europe',
    'United States': 'North America',
    'Uruguay': 'South America',
    'Uzbekistan': 'Asia',
    'Vanuatu': 'Oceania',
    'Venezuela (Bolivarian Republic of)': 'South America',
    'Viet Nam': 'Asia',
    'Yemen': 'Asia',
    'Yemen Arab Republic (Former)': 'Asia',
    'Yemen Democratic (Former)': 'Asia',
    'Yugoslavia (Former)': 'Europe',
    'Zambia': 'Africa',
    'Zanzibar': 'Africa',
    'Zimbabwe': 'Africa'
}

df['continent'] = df['country'].map(country_to_continent)

In [11]:
# Define the country code map with year ranges for former countries
country_code_map = {
    'Afghanistan': 'AFG', 'Albania': 'ALB', 'Algeria': 'DZA', 'Andorra': 'AND', 'Angola': 'AGO', 'Anguilla': 'AIA', 
    'Antigua and Barbuda': 'ATG', 'Argentina': 'ARG', 'Armenia': 'ARM', 'Aruba': 'ABW', 'Australia': 'AUS', 
    'Austria': 'AUT', 'Azerbaijan': 'AZE', 'Bahamas': 'BHS', 'Bahrain': 'BHR', 'Bangladesh': 'BGD', 'Barbados': 'BRB', 
    'Belarus': 'BLR', 'Belgium': 'BEL', 'Belize': 'BLZ', 'Benin': 'BEN', 'Bermuda': 'BMU', 'Bhutan': 'BTN', 
    'Bolivia (Plurinational State of)': 'BOL', 'Bosnia and Herzegovina': 'BIH', 'Botswana': 'BWA', 'Brazil': 'BRA', 
    'British Virgin Islands': 'VGB', 'Brunei Darussalam': 'BRN', 'Bulgaria': 'BGR', 'Burkina Faso': 'BFA', 
    'Burundi': 'BDI', 'Cabo Verde': 'CPV', 'Cambodia': 'KHM', 'Cameroon': 'CMR', 'Canada': 'CAN', 
    'Cayman Islands': 'CYM', 'Central African Republic': 'CAF', 'Chad': 'TCD', 'Chile': 'CHL', 'China': 'CHN', 
    'China, Hong Kong SAR': 'HKG', 'China, Macao SAR': 'MAC', 'Colombia': 'COL', 'Comoros': 'COM', 'Congo': 'COG', 
    'Cook Islands': 'COK', 'Costa Rica': 'CRI', 'Croatia': 'HRV', 'Cuba': 'CUB', 'Curaçao': 'CUW', 'Cyprus': 'CYP', 
    'Czechia': 'CZE', "Côte d'Ivoire": 'CIV', 'D.P.R. of Korea': 'PRK', 'D.R. of the Congo': 'COD', 'Denmark': 'DNK', 
    'Djibouti': 'DJI', 'Dominica': 'DMA', 'Dominican Republic': 'DOM', 'Ecuador': 'ECU', 'Egypt': 'EGY', 
    'El Salvador': 'SLV', 'Equatorial Guinea': 'GNQ', 'Eritrea': 'ERI', 'Estonia': 'EST', 'Eswatini': 'SWZ', 
    'Ethiopia': 'ETH', 'Ethiopia (Former)': {'code': 'ETH', 'start_year': None, 'end_year': 1991}, 'Fiji': 'FJI', 
    'Finland': 'FIN', 'Former Netherlands Antilles': {'code': 'ANT', 'start_year': None, 'end_year': 2010}, 
    'France': 'FRA', 'French Polynesia': 'PYF', 'Gabon': 'GAB', 'Gambia': 'GMB', 'Georgia': 'GEO', 'Germany': 'DEU', 
    'Ghana': 'GHA', 'Greece': 'GRC', 'Greenland': 'GRL', 'Grenada': 'GRD', 'Guatemala': 'GTM', 'Guinea': 'GIN', 
    'Guinea-Bissau': 'GNB', 'Guyana': 'GUY', 'Haiti': 'HTI', 'Honduras': 'HND', 'Hungary': 'HUN', 'Iceland': 'ISL', 
    'India': 'IND', 'Indonesia': 'IDN', 'Iran (Islamic Republic of)': 'IRN', 'Iraq': 'IRQ', 'Ireland': 'IRL', 
    'Israel': 'ISR', 'Italy': 'ITA', 'Jamaica': 'JAM', 'Japan': 'JPN', 'Jordan': 'JOR', 'Kazakhstan': 'KAZ', 
    'Kenya': 'KEN', 'Kiribati': 'KIR', 'Kosovo': 'XKX', 'Kuwait': 'KWT', 'Kyrgyzstan': 'KGZ', "Lao People's DR": 'LAO', 
    'Latvia': 'LVA', 'Lebanon': 'LBN', 'Lesotho': 'LSO', 'Liberia': 'LBR', 'Libya': 'LBY', 'Liechtenstein': 'LIE', 
    'Lithuania': 'LTU', 'Luxembourg': 'LUX', 'Madagascar': 'MDG', 'Malawi': 'MWI', 'Malaysia': 'MYS', 'Maldives': 'MDV', 
    'Mali': 'MLI', 'Malta': 'MLT', 'Marshall Islands': 'MHL', 'Mauritania': 'MRT', 'Mauritius': 'MUS', 'Mexico': 'MEX', 
    'Micronesia (FS of)': 'FSM', 'Monaco': 'MCO', 'Mongolia': 'MNG', 'Montenegro': 'MNE', 'Montserrat': 'MSR', 
    'Morocco': 'MAR', 'Mozambique': 'MOZ', 'Myanmar': 'MMR', 'Namibia': 'NAM', 'Nauru': 'NRU', 'Nepal': 'NPL', 
    'Netherlands': 'NLD', 'New Caledonia': 'NCL', 'New Zealand': 'NZL', 'Nicaragua': 'NIC', 'Niger': 'NER', 
    'Nigeria': 'NGA', 'North Macedonia': 'MKD', 'Norway': 'NOR', 'Oman': 'OMN', 'Pakistan': 'PAK', 'Palau': 'PLW', 
    'Panama': 'PAN', 'Papua New Guinea': 'PNG', 'Paraguay': 'PRY', 'Peru': 'PER', 'Philippines': 'PHL', 'Poland': 'POL', 
    'Portugal': 'PRT', 'Puerto Rico': 'PRI', 'Qatar': 'QAT', 'Republic of Korea': 'KOR', 'Republic of Moldova': 'MDA', 
    'Romania': 'ROU', 'Russian Federation': 'RUS', 'Rwanda': 'RWA', 'Saint Kitts and Nevis': 'KNA', 'Saint Lucia': 'LCA', 
    'Samoa': 'WSM', 'San Marino': 'SMR', 'Sao Tome and Principe': 'STP', 'Saudi Arabia': 'SAU', 'Senegal': 'SEN', 
    'Serbia': 'SRB', 'Seychelles': 'SYC', 'Sierra Leone': 'SLE', 'Singapore': 'SGP', 'Sint Maarten (Dutch part)': 'SXM', 
    'Slovakia': 'SVK', 'Slovenia': 'SVN', 'Solomon Islands': 'SLB', 'Somalia': 'SOM', 'South Africa': 'ZAF', 
    'South Sudan': 'SSD', 'Spain': 'ESP', 'Sri Lanka': 'LKA', 'St. Vincent and the Grenadines': 'VCT', 
    'State of Palestine': 'PSE', 'Sudan': 'SDN', 'Sudan (Former)': {'code': 'SDN', 'start_year': None, 'end_year': 2011}, 
    'Suriname': 'SUR', 'Sweden': 'SWE', 'Switzerland': 'CHE', 'Syrian Arab Republic': 'SYR', 'Tajikistan': 'TJK', 
    'Thailand': 'THA', 'Timor-Leste': 'TLS', 'Togo': 'TGO', 'Tonga': 'TON', 'Trinidad and Tobago': 'TTO', 'Tunisia': 'TUN', 
    'Turkmenistan': 'TKM', 'Turks and Caicos Islands': 'TCA', 'Tuvalu': 'TUV', 'Türkiye': 'TUR', 
    'U.R. of Tanzania: Mainland': 'TZA', 'USSR (Former)': {'code': 'SUN', 'start_year': None, 'end_year': 1991}, 
    'Uganda': 'UGA', 'Ukraine': 'UKR', 'United Arab Emirates': 'ARE', 'United Kingdom': 'GBR', 'United States': 'USA', 
    'Uruguay': 'URY', 'Uzbekistan': 'UZB', 'Vanuatu': 'VUT', 'Venezuela (Bolivarian Republic of)': 'VEN', 'Viet Nam': 'VNM', 
    'Yemen': 'YEM', 'Yemen Arab Republic (Former)': {'code': 'YAR', 'start_year': None, 'end_year': 1990}, 
    'Yemen Democratic (Former)': {'code': 'YDR', 'start_year': None, 'end_year': 1990}, 
    'Yugoslavia (Former)': {'code': 'YUG', 'start_year': None, 'end_year': 2003}, 'Zambia': 'ZMB', 'Zanzibar': '', 
    'Zimbabwe': 'ZWE'
}

# Define the function to get the correct ISO3 code
def get_iso3_code(country, year):
    if country not in country_code_map:
        return None
    
    info = country_code_map[country]
    
    if isinstance(info, str):
        return info
    
    # Check for former countries with year ranges
    if isinstance(info, dict):
        if (info['start_year'] is None or info['start_year'] <= year) and (info['end_year'] is None or info['end_year'] >= year):
            return info['code']
        
    return None

# Apply the function to your DataFrame
df['code_iso3'] = df.apply(lambda row: get_iso3_code(row['country'], row['year']), axis=1)

In [12]:
# get the modern iso3-country-codes for former countries
historical_to_modern_iso3 = {
    'SUN': 'RUS',  # Mapping USSR to Russia
    'YUG': 'SRB',  # Mapping Yugoslavia to Serbia (could be multiple countries)
    'YAR': 'YEM',  # Mapping Yemen Arab Republic to modern Yemen
    'YDR': 'YEM',  # Mapping Yemen Democratic Republic to modern Yemen
    'ANT': 'CUW'   # Mapping Netherlands Antilles to Curaçao
}

# Replace historical ISO3 codes with modern equivalents in the DataFrame
df['code_iso3'] = df['code_iso3'].replace(historical_to_modern_iso3)

In [13]:
# Define the country to region mapping using ISO3 codes, including former countries
iso3_to_region = {
    # Africa
    'DZA': 'Northern Africa', 'EGY': 'Northern Africa', 'LBY': 'Northern Africa', 'MAR': 'Northern Africa',
    'SDN': 'Northern Africa', 'TUN': 'Northern Africa', 'ESH': 'Northern Africa',

    'BDI': 'Eastern Africa', 'COM': 'Eastern Africa', 'DJI': 'Eastern Africa', 'ERI': 'Eastern Africa',
    'ETH': 'Eastern Africa', 'KEN': 'Eastern Africa', 'MDG': 'Eastern Africa', 'MUS': 'Eastern Africa',
    'RWA': 'Eastern Africa', 'SYC': 'Eastern Africa', 'SOM': 'Eastern Africa', 'SSD': 'Eastern Africa',
    'TZA': 'Eastern Africa', 'UGA': 'Eastern Africa', 'MWI': 'Eastern Africa', 'MOZ': 'Eastern Africa',
    'ZMB': 'Eastern Africa', 'ZWE': 'Eastern Africa',

    'AGO': 'Middle Africa', 'CMR': 'Middle Africa', 'CAF': 'Middle Africa', 'TCD': 'Middle Africa',
    'COG': 'Middle Africa', 'COD': 'Middle Africa', 'GNQ': 'Middle Africa', 'GAB': 'Middle Africa',
    'STP': 'Middle Africa',

    'BEN': 'Western Africa', 'BFA': 'Western Africa', 'CPV': 'Western Africa', 'CIV': 'Western Africa',
    'GMB': 'Western Africa', 'GHA': 'Western Africa', 'GIN': 'Western Africa', 'GNB': 'Western Africa',
    'LBR': 'Western Africa', 'MLI': 'Western Africa', 'MRT': 'Western Africa', 'NER': 'Western Africa',
    'NGA': 'Western Africa', 'SEN': 'Western Africa', 'SLE': 'Western Africa', 'TGO': 'Western Africa',

    'BWA': 'Southern Africa', 'SWZ': 'Southern Africa', 'LSO': 'Southern Africa', 'NAM': 'Southern Africa',
    'ZAF': 'Southern Africa',

    # Americas
    'CAN': 'Northern America', 'USA': 'Northern America', 'GRL': 'Northern America', 'BMU': 'Northern America',
    'SPM': 'Northern America',

    'BLZ': 'Central America', 'CRI': 'Central America', 'SLV': 'Central America', 'GTM': 'Central America',
    'HND': 'Central America', 'NIC': 'Central America', 'PAN': 'Central America', 'MEX': 'Central America',

    'AIA': 'Caribbean', 'ATG': 'Caribbean', 'ABW': 'Caribbean', 'BHS': 'Caribbean', 'BRB': 'Caribbean',
    'VGB': 'Caribbean', 'CYM': 'Caribbean', 'CUB': 'Caribbean', 'CUW': 'Caribbean', 'DMA': 'Caribbean',
    'DOM': 'Caribbean', 'GRD': 'Caribbean', 'GLP': 'Caribbean', 'HTI': 'Caribbean', 'JAM': 'Caribbean',
    'MTQ': 'Caribbean', 'MSR': 'Caribbean', 'PRI': 'Caribbean', 'BLM': 'Caribbean', 'KNA': 'Caribbean',
    'LCA': 'Caribbean', 'MAF': 'Caribbean', 'VCT': 'Caribbean', 'SXM': 'Caribbean', 'TTO': 'Caribbean',
    'TCA': 'Caribbean', 'VIR': 'Caribbean',

    'ARG': 'South America', 'BOL': 'South America', 'BRA': 'South America', 'CHL': 'South America',
    'COL': 'South America', 'ECU': 'South America', 'FLK': 'South America', 'GUF': 'South America',
    'GUY': 'South America', 'PRY': 'South America', 'PER': 'South America', 'SUR': 'South America',
    'URY': 'South America', 'VEN': 'South America',

    # Asia
    'CHN': 'Eastern Asia', 'HKG': 'Eastern Asia', 'MAC': 'Eastern Asia', 'JPN': 'Eastern Asia',
    'KOR': 'Eastern Asia', 'MNG': 'Eastern Asia', 'PRK': 'Eastern Asia',

    'AFG': 'Southern Asia', 'BGD': 'Southern Asia', 'BTN': 'Southern Asia', 'IND': 'Southern Asia',
    'IRN': 'Southern Asia', 'MDV': 'Southern Asia', 'NPL': 'Southern Asia', 'PAK': 'Southern Asia',
    'LKA': 'Southern Asia',

    'BRN': 'South-Eastern Asia', 'KHM': 'South-Eastern Asia', 'IDN': 'South-Eastern Asia', 'LAO': 'South-Eastern Asia',
    'MYS': 'South-Eastern Asia', 'MMR': 'South-Eastern Asia', 'PHL': 'South-Eastern Asia', 'SGP': 'South-Eastern Asia',
    'THA': 'South-Eastern Asia', 'TLS': 'South-Eastern Asia', 'VNM': 'South-Eastern Asia',

    'ARM': 'Western Asia', 'AZE': 'Western Asia', 'BHR': 'Western Asia', 'CYP': 'Western Asia', 'GEO': 'Western Asia',
    'IRQ': 'Western Asia', 'ISR': 'Western Asia', 'JOR': 'Western Asia', 'KWT': 'Western Asia', 'LBN': 'Western Asia',
    'OMN': 'Western Asia', 'PSE': 'Western Asia', 'QAT': 'Western Asia', 'SAU': 'Western Asia', 'SYR': 'Western Asia',
    'ARE': 'Western Asia', 'YEM': 'Western Asia', 'TUR': 'Western Asia',

    'KAZ': 'Central Asia', 'KGZ': 'Central Asia', 'TJK': 'Central Asia', 'TKM': 'Central Asia', 'UZB': 'Central Asia',

    # Europe
    'AND': 'Southern Europe', 'AUT': 'Western Europe', 'BEL': 'Western Europe', 'FRA': 'Western Europe',
    'DEU': 'Western Europe', 'IRL': 'Northern Europe', 'LIE': 'Western Europe', 'LUX': 'Western Europe',
    'MCO': 'Western Europe', 'NLD': 'Western Europe', 'CHE': 'Western Europe', 'GBR': 'Northern Europe',
    'DNK': 'Northern Europe', 'FIN': 'Northern Europe', 'GRC': 'Southern Europe', 'ISL': 'Northern Europe',
    'ITA': 'Southern Europe', 'SMR': 'Southern Europe', 'ESP': 'Southern Europe', 'SWE': 'Northern Europe',
    'NOR': 'Northern Europe', 'PRT': 'Southern Europe', 'MLT': 'Southern Europe',

    'ALB': 'Southern Europe', 'BLR': 'Eastern Europe', 'BIH': 'Southern Europe', 'BGR': 'Eastern Europe',
    'HRV': 'Southern Europe', 'CZE': 'Eastern Europe', 'EST': 'Northern Europe', 'HUN': 'Eastern Europe',
    'XKX': 'Southern Europe', 'LVA': 'Northern Europe', 'LTU': 'Northern Europe', 'MKD': 'Southern Europe',
    'POL': 'Eastern Europe', 'MDA': 'Eastern Europe', 'ROU': 'Eastern Europe', 'RUS': 'Eastern Europe',
    'SRB': 'Southern Europe', 'SVK': 'Eastern Europe', 'SVN': 'Southern Europe', 'UKR': 'Eastern Europe',
    'MNE': 'Southern Europe',

    # Oceania
    'AUS': 'Australia and New Zealand', 'NZL': 'Australia and New Zealand',

    'FJI': 'Melanesia', 'PNG': 'Melanesia', 'SLB': 'Melanesia', 'VUT': 'Melanesia',

    'FSM': 'Micronesia', 'PLW': 'Micronesia', 'KIR': 'Micronesia', 'MHL': 'Micronesia', 'NRU': 'Micronesia',

    'WSM': 'Polynesia', 'TON': 'Polynesia', 'TUV': 'Polynesia', 'COK': 'Polynesia', 'PYF': 'Polynesia',
    'NIU': 'Polynesia', 'TKL': 'Polynesia', 'WLF': 'Polynesia',

    # Former countries
    'SUN': 'Eastern Europe', 'YUG': 'Southern Europe', 'YAR': 'Western Asia', 'YDR': 'Western Asia', 'ANT': 'Caribbean', 

}

df['region'] = df['code_iso3'].map(iso3_to_region)

In [14]:
# combine the regions to bigger regions

def combine_regions(region):
    if region in ['Southern Europe', 'Northern Europe', 'Western Europe']:
        return 'Western Europe'
    elif region in ['Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia']:
        return 'Oceania'
    elif region in ['Central America', 'Caribbean']:
        return 'Central America'
    elif region in ['Eastern Africa', 'Southern Africa']:
        return 'Eastern and Southern Africa'
    elif region in ['Western Africa', 'Middle Africa']:
        return 'Western and Central Africa'
    else:
        return region


df['region'] = df['region'].apply(combine_regions)

In [15]:
# create a dataframe just for 2018
df_2018 = df.query('year == 2018').copy()

In [16]:
# drop rows with missing values
df_2018.dropna(subset='score', inplace=True)
df_2018.dropna(subset='emission', inplace=True)
df_2018.dropna(subset='percepted_corruption', inplace=True)

In [17]:
h_agg_df = df_2018.groupby('region').agg({'score': 'mean', 
                                     'social_support': 'mean', 
                                     'healthy_life_expectancy': 'mean', 
                                     'freedom_life_choices': 'mean', 
                                     'generosity': 'mean', 
                                     'percepted_corruption': 'mean'
                                     }).reset_index()

In [18]:
# sort happiness score descending
sorted_score_df = h_agg_df.sort_values(by='score', ascending=False)

In [19]:
m_agg_df = df_2018.groupby('region').agg({'score': 'mean', 
                                     'emission': 'mean', 
                                     'per_capita_gni': 'mean', 
                                     'gdp_capita': 'mean', 
                                     'p_agriculture,_hunting,_forestry,_fishing': 'mean', 
                                     'p_other_activities': 'mean',
                                     'life_expectancy': 'mean',
                                     'pop_density': 'mean',
                                     'population': 'sum'
                                     }).reset_index()

In [24]:

# aggregate the dataframe by region and continent
aggregated_df = df.groupby(['region', 'continent']).agg({
    'emission': 'mean',
    'gdp_capita': 'mean',
    'population': 'mean',
    'life_expectancy': 'mean',
    'pop_density': 'mean'
}).reset_index()

# aggregate the dataframe by region and year
aggregated_df_year = df.groupby(['region', 'year']).agg({
    'emission': 'mean',
    'gdp_capita': 'mean',
    'population': 'mean',
    'life_expectancy': 'mean',
    'emission_total': 'mean',
    'gross_domestic_product_(gdp)': 'mean',
    'p_agriculture,_hunting,_forestry,_fishing' : 'mean',
    'p_mining_manufacturing_utilities': 'mean',
    'p_other_activities': 'mean',
    'p_wholesale_trade_restaurants': 'mean',
    'p_transport,_storage_and_communication_(isic_i)': 'mean',
    'p_general_government_final_consumption_expenditure': 'mean',
    'p_household_consumption_expenditure': 'mean'
}).reset_index()

region_color_map = {
    'Southern Asia': '#0d3b66',         # Dark Blue
    'Western Europe': '#f4a261',        # Matte Orange
    'Northern Africa': '#2a9d8f',       # Dark Teal
    'Western and Central Africa': '#d8574e', # Dark Red
    'Central America': '#3b884d',       # Matte Light Brown
    'South America': '#8d3b72',         # Matte Burgundy
    'Western Asia': '#bc6c25',          # Matte Brown
    'Oceania': '#495867',               # Dark Grey
    'Eastern Europe': '#ff6f61',        # Matte Coral
    'Northern America': '#206a5d',      # Dark Green
    'Eastern and Southern Africa': '#3d5a80', # Dark Slate
    'South-Eastern Asia': '#c8553d',    # Matte Brick Red
    'Eastern Asia': '#817c9f',          # Matte Plum
    'Central Asia': '#986B6B',          # Matte Peach
    '': '#7f7f7f'                       # Default Grey for NaN values
}


region_color_list = list(region_color_map.values())

# Build the app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SKETCHY])
load_figure_template("sketchy")



app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Data Muse"), style={'font-size': '4rem'}, className='text-center')
    ]),
    dbc.Row([
        dbc.Col(html.H3("Inspiring the Author"), className='text-center') 
    ]),
    dbc.Row(dbc.Col(html.H3("Our World in Numbers"), width=12)),
    dbc.Row([
        dbc.Col(dcc.Graph(id='choropleth-map'), width=5),
        dbc.Col(dcc.Graph(id='lines-overview'), width=7)
    ]),
    dbc.Row([
        dbc.Col(
            dcc.Dropdown(
                id='yaxis-dropdown',
                options=[
                {'label': 'GDP per Capita', 'value': 'gdp_capita'},
                {'label': 'Mean Emissions per Capita', 'value': 'emission'},
                {'label': 'Life Expectancy', 'value': 'life_expectancy'},
                {'label': '%Primary Sector in GDP', 'value': 'p_agriculture,_hunting,_forestry,_fishing'},
                {'label': '%Secondary Sector in GDP', 'value': 'p_mining_manufacturing_utilities'},
                {'label': '%Tertiary Sector in GDP', 'value': 'p_other_activities'},
                {'label': 'Population density', 'value': 'pop_density'},
                {'label': 'Population', 'value': 'population'},
                {'label': 'Emissions total', 'value': 'emission_total'},
                {'label': 'Emissions in Relation to GDP', 'value': 'emission_gdp'}
                ],
                value='gdp_capita',
                clearable=False
            ),
        width={'size': 4, 'offset': 7}
        )
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='choropleth-map_2'), width=7),
        dbc.Col(dcc.Graph(id="differences-bar"), width=5),
    ]),
    dbc.Row([
        dbc.Col(
            dcc.Dropdown(
                id='value-dropdown_2',
                options=[
                {'label': 'GDP per Capita', 'value': 'gdp_capita'},
                {'label': 'Mean Emissions per Capita', 'value': 'emission'},
                {'label': 'Life Expectancy', 'value': 'life_expectancy'},
                {'label': '%Primary Sector in GDP', 'value': 'p_agriculture,_hunting,_forestry,_fishing'},
                {'label': '%Secondary Sector in GDP', 'value': 'p_mining_manufacturing_utilities'},
                {'label': '%Tertiary Sector in GDP', 'value': 'p_other_activities'},
                {'label': 'Population density', 'value': 'pop_density'},
                {'label': 'Population', 'value': 'population'},
                {'label': 'Emissions total', 'value': 'emission_total'},
                {'label': 'Emissions in Relation to GDP', 'value': 'emission_gdp'}
                ],
                value='gdp_capita',
                clearable=False
            ), width=7),
        dbc.Col(dcc.Dropdown(
            id='xaxis-dropdown_2',
            options=[
                {'label': 'GDP per Capita', 'value': 'gdp_capita'},
                {'label': 'Mean Emissions per Capita', 'value': 'emission'},
                {'label': 'Life Expectancy', 'value': 'life_expectancy'},
                {'label': '%Primary Sector in GDP', 'value': 'p_agriculture,_hunting,_forestry,_fishing'},
                {'label': '%Secondary Sector in GDP', 'value': 'p_mining_manufacturing_utilities'},
                {'label': '%Tertiary Sector in GDP', 'value': 'p_other_activities'},
                {'label': 'Population density', 'value': 'pop_density'},
                {'label': 'Population', 'value': 'population'},
                {'label': 'Emissions total', 'value': 'emission_total'},
                {'label': 'Emissions in Relation to GDP', 'value': 'emission_gdp'}
            ],
            value='emission',
            clearable=False
        ), width=5)
        ]),
    dbc.Row(),
    dbc.Row(dbc.Col(
        dcc.Slider(
            id='year-slider_2',
            min=df['year'].min(),
            max=df['year'].max(),
            value=df['year'].min(),
            marks={str(year): str(year) for year in df['year'].unique() if year % 5 == 0},
            step=1
        ), width=12, style={'padding':'20px'}
    )),
    dbc.Row(dbc.Col(html.H3("Getting a closer look at the Regions"), width=12)),
    dbc.Row([
        dbc.Col(
            dcc.Dropdown(
                id='region-select',
                options=[{'label': region, 'value': region} for region in aggregated_df_year['region'].unique()],
                value=aggregated_df_year['region'].iloc[0],
                clearable=False,
            ), width=4
        ),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='line-plot_region'), width=4),
        dbc.Col(dcc.Graph(id='line-plot_region2'), width=4),
        dbc.Col(dcc.Graph(id='line-plot_region3'), width=4)    
    ]),
    dbc.Row(dbc.Col(html.H3("Regional Happiness in 2018"), width=12)),
    dbc.Row(dbc.Col(dcc.Graph(id='bar-chart_happ', style={'flex': '1'}))),
    dbc.Row(dcc.Graph(id='scatter-plot')),
    dbc.Row(dbc.Col(
        dcc.Dropdown(
        id='yaxis-column',
        options=[                
                {'label': 'GDP per Capita', 'value': 'gdp_capita'},
                {'label': 'Mean Emissions per Capita', 'value': 'emission'},
                {'label': 'Life Expectancy', 'value': 'life_expectancy'},
                {'label': '%Primary Sector in GDP', 'value': 'p_agriculture,_hunting,_forestry,_fishing'},
                {'label': '%Secondary Sector in GDP', 'value': 'p_mining_manufacturing_utilities'},
                {'label': '%Tertiary Sector in GDP', 'value': 'p_other_activities'},
                {'label': 'Population density', 'value': 'pop_density'},
                ],
        value='gdp_capita',
        clearable=False,
        style={'marginBottom': '220px'}
        ), width = 6)
    )
], fluid=True, className="dbc")


# first map
@app.callback(
    Output('choropleth-map', 'figure'),
    [Input('choropleth-map', 'id')] 
)
def update_choropleth_figure(_):
    # Filter the DataFrame to only include the year 2018
    df_2018_2 = df[df['year'] == 2018].copy()
    df_2018_2['hover_text'] = df_2018_2['region'] + '<br>' + df_2018_2['country']
    
    fig = px.choropleth(df_2018_2,
                        locations='code_iso3',
                        color='region', 
                        hover_name=df_2018_2['hover_text'],
                        custom_data=['region', 'country'],
                        projection='natural earth',
                        color_discrete_sequence=list(region_color_map.values())) 
    fig.update_traces(
    hovertemplate='<b>Region:</b> %{customdata[0]}<br>' +
                  '<b>Country:</b> %{customdata[1]}<extra></extra>'
        )
    fig.update_layout(
        showlegend=False,
        paper_bgcolor='rgba(0, 0, 0, 0)',
        plot_bgcolor='rgba(0, 0, 0, 0)',
        geo=dict(bgcolor='rgba(0,0,0,0)', 
                 projection=dict(type='natural earth', scale=1.09),
                 center=dict(lat=7, lon=0),
                 domain=dict(x=[0, 1], y=[0, 1])),
                 )

    return fig

# line-plot to show development over time
@app.callback(
    Output('lines-overview', 'figure'),
    Input('yaxis-dropdown', 'value')
)
def update_line_plot(selected_value):
    # Aggregate the DataFrame by year and region, taking the mean of the selected value
    aggregated_df = df.groupby(['year', 'region'])[selected_value].mean().reset_index()
    
    # Calculate the mean value across all regions for each year
    mean_df = df.groupby('year')[selected_value].mean().reset_index()
    mean_df['region'] = 'World Mean'  # Add a region label for the mean line

    # Combine the aggregated data with the mean data
    combined_df = pd.concat([aggregated_df, mean_df], ignore_index=True)
    
    max_value = combined_df[selected_value].max()
    min_value = combined_df[selected_value].min()

    # Create a line plot for all regions and the mean line
    fig = px.line(combined_df, x='year', y=selected_value, color='region',
                  color_discrete_map={**region_color_map, 'World Mean': 'rgb(50, 50, 50)'})
    
    fig.update_traces(line=dict(width=3), visible='legendonly')
    
    fig.update_layout(
        paper_bgcolor='rgba(0, 0, 0, 0)',
        plot_bgcolor='rgba(0, 0, 0, 0)',
        legend=dict(
            x=-0.6,  
            y=0.5,   
            font=dict(
                size=16,
                color="rgb(50, 50, 50)"
            ),
            bgcolor='rgba(0, 0, 0, 0)', 
            bordercolor='rgba(0, 0, 0, 0)',
            title='Regions',
            itemsizing='constant',
            traceorder='normal',
            borderwidth=1
        ),
        height=535,
        yaxis=dict(range=[min_value, max_value],  tickfont=dict(size=14, color="rgb(50, 50, 50)")),
        xaxis=dict(range=[1970, 2018], tickfont=dict(size=14, color="rgb(50, 50, 50)")),
        yaxis_title='',
        xaxis_title=''
    ),
    

    # Make the average line always visible
    for trace in fig.data:
        if trace.name == 'World Mean':
            trace.visible = True

    return fig



# Define callback to update the choropleth map based on the selected year and value
@app.callback(
    Output('choropleth-map_2', 'figure'),
    [Input('year-slider_2', 'value'),
     Input('value-dropdown_2', 'value')]
)


def update_choropleth_map(selected_year, selected_value):
    # Filter the DataFrame for the selected year
    filtered_df = df[df['year'] == selected_year]
    
    # Group the filtered dataframe by region and calculate the mean of the selected value
    region_mean_values = filtered_df.groupby('region')[selected_value].mean().reset_index()
    
    # Create a mapping from country to region
    country_region_map = df[['country', 'region', 'code_iso3']].drop_duplicates()
    
    # Merge the regional mean values with the country_region_map to assign region values to countries
    merged_df = pd.merge(country_region_map, region_mean_values, on='region', how='left')
    
    # Set the range color to the min and max mean values
    range_color = [region_mean_values[selected_value].min(), region_mean_values[selected_value].max()]

    selected_label = next(option['label'] for option in [
    {'label': 'GDP per Capita', 'value': 'gdp_capita'},
    {'label': 'Mean Emissions per Capita', 'value': 'emission'},
    {'label': 'Life Expectancy', 'value': 'life_expectancy'},
    {'label': '%Primary Sector in GDP', 'value': 'p_agriculture,_hunting,_forestry,_fishing'},
    {'label': '%Secondary Sector in GDP', 'value': 'p_mining_manufacturing_utilities'},
    {'label': '%Tertiary Sector in GDP', 'value': 'p_other_activities'},
    {'label': 'Population density', 'value': 'pop_density'},
    {'label': 'Population', 'value': 'population'},
    {'label': 'Emissions total', 'value': 'emission_total'},
    {'label': 'Emissions in Relation to GDP', 'value': 'emission_gdp'}
        ] if option['value'] == selected_value)
    
    fig = px.choropleth(
        merged_df,
        locations='code_iso3',
        hover_name='country',
        color=selected_value,
        projection='natural earth',
        range_color=range_color
    )
    
    fig.update_geos(showcountries=True, countrycolor="Grey", showcoastlines=True)

    fig.update_traces(hovertemplate=
                                 '<b>Region: </b>' + merged_df['region'] + '<br>' +
                                '<b>Country:</b> %{hovertext}<br>' +
                                 f'<b>{selected_label}:</b> %{{z}}<extra></extra>')
    fig.update_layout(
        coloraxis_colorbar=dict(
            x=0,  
            y=0.5,  
            len=1,
            title=''
        ),
        paper_bgcolor='rgba(0, 0, 0, 0)',
        plot_bgcolor='rgba(0, 0, 0, 0)',
        geo=dict(bgcolor='rgba(0,0,0,0)', 
                 projection=dict(type='natural earth', scale=1.09),
                 center=dict(lat=7, lon=0),
                 domain=dict(x=[0, 1], y=[0, 1])),
    )
    
    return fig

# Define the bar plot to see differences between the regions
@app.callback(
    Output('differences-bar', 'figure'),
    [Input('xaxis-dropdown_2', 'value'),
     Input('year-slider_2', 'value')]
)
def create_bar1(selected_x_2, selected_year):
    # Filter the aggregated df by the selected year, sort it descending by selected value and calculate world mean
    filtered_df = aggregated_df_year[aggregated_df_year['year'] == selected_year]
    filtered_df = filtered_df.sort_values(by=selected_x_2, ascending=False)

    mean_x_world = filtered_df[selected_x_2].mean()

    

    fig = px.bar(
        filtered_df,
        x='region',
        y=selected_x_2,
        color='region',
        color_discrete_map=region_color_map  
    )
    fig.update_traces(hovertemplate='<b>%{x}</b><br>' +
                                    f'{selected_x_2.replace("_", " ").title()}: %{{y}}<extra></extra>')
    fig.update_layout(
        paper_bgcolor='rgba(0, 0, 0, 0)',
        plot_bgcolor='rgba(0, 0, 0, 0)',
        showlegend=False,
        xaxis=dict(tickfont=dict(size=14)),
        yaxis=dict(tickfont=dict(size=14)),
        yaxis_title='',
        xaxis_title=''
    )

    # Add a horizontal line for the world mean
    fig.add_hline(
        y=mean_x_world,
        line_dash="dash",
        line_color="rgb(50, 50, 50)",
        annotation_text=f"<span style='font-size: 14px'>World Mean: {mean_x_world:.2f}",
        annotation_position="top right"
    )

    return fig

# Define the callback to update the bar chart for happiness
@app.callback(
    Output('bar-chart_happ', 'figure'),
    [Input('bar-chart_happ', 'id')]  
)
def update_bar_chart(_):
    fig = go.Figure()

    # Add the 'score' bar (always shown) on the primary y-axis
    fig.add_trace(go.Bar(
        x=sorted_score_df['region'],
        y=sorted_score_df['score'],
        name='Score',
        width=0.6,  
        yaxis='y1', 
        marker_color=[region_color_map[region] for region in sorted_score_df['region']]  
    ))

    # Add the selected columns as lines on the secondary y-axis
    for column in ['social_support', 'healthy_life_expectancy', 'freedom_life_choices', 'generosity', 'percepted_corruption']:
        fig.add_trace(go.Scatter(
            x=sorted_score_df['region'],
            y=sorted_score_df[column],
            name=column.replace('_', ' ').title(),
            mode='lines+markers',
            yaxis='y2',
            visible='legendonly',
            line=dict(width=3),
            marker=dict(size=12, line=dict(width=1, color='white'))
        ))

    fig.update_layout(
        barmode='overlay',  
        yaxis_title='Score',
        yaxis=dict(range=[0, 10], title='Happiness-Score', titlefont=dict(size=16), side='left', title_standoff=10, tickfont=dict(size=14)), 
        yaxis2=dict(range=[0, 2], title='Other Metrics', titlefont=dict(size=16), overlaying='y', side='right', showgrid=False, title_standoff=10, tickfont=dict(size=14)),  
        legend_title_text='Metrics',
        legend=dict(x=1.05), 
        xaxis={'tickvals': list(range(len(sorted_score_df['region']))), 'ticktext': sorted_score_df['region'], 'range': [-0.5, 14.5], 'tickfont':dict(size=16)}, 
        plot_bgcolor='rgba(0,0,0,0)',  
        paper_bgcolor='rgba(0,0,0,0)'
    )

    return fig

# Callback to create the scatter plot for happiness
@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('yaxis-column', 'value')],
    [State('yaxis-column', 'options')]
)
def update_graph(yaxis_column_name, dropdown_options):

    yaxis_label = next(option['label'] for option in dropdown_options if option['value'] == yaxis_column_name)
    fig = px.scatter(
        m_agg_df, 
        x='score', 
        y=yaxis_column_name, 
        color='region', 
        size='population', 
        size_max=60,
        custom_data=['region'],  
        color_discrete_map=region_color_map  
    )
    
    # Customize hover data format
    fig.update_traces(
        hovertemplate=(
            '<b>Region:</b> %{customdata[0]}<br>'
            '<b>Score:</b> %{x:,.2f}<br>'
            '<b>' + yaxis_column_name + ':</b> %{y:,.2f}<br>'
            '<b>Population:</b> %{marker.size:,.0f}<extra></extra>'
        ),
        marker=dict(opacity=1)
    )
    # Calculate overall mean for selected x-axis value
    overall_mean_y = m_agg_df[yaxis_column_name].mean()
    overall_mean_score = m_agg_df['score'].mean()


 # Add overall mean lines
    fig.add_vline(
        x=overall_mean_score,
        line_dash="dash",
        line_color="rgb(50, 50, 50)",
        annotation_text=f"<span style='font-size: 14px'>Overall Happiness-score: {overall_mean_score:.2f}",
        annotation_position="bottom right"
    )
    fig.add_hline(
        y=overall_mean_y,
        line_dash="dash",
        line_color="rgb(50, 50, 50)",
        annotation_text=f"<span style='font-size: 14px'>Overall Mean {yaxis_label}: {overall_mean_y:.2f}",
        annotation_position="top right"
    )
    fig.update_layout(
        showlegend=True, 
        xaxis=dict(range=[3.5, 7.5], title='Happiness-Score', titlefont=dict(size=16), side='left', title_standoff=10, tickfont=dict(size=14)),
        yaxis=dict(titlefont=dict(size=16), side='left', title_standoff=10, tickfont=dict(size=14)),
        yaxis_title=yaxis_label,
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)'
    )
    return fig

@app.callback(
    Output('line-plot_region', 'figure'),
    [Input('region-select', 'value')]
)
def update_line_plot(selected_region):
    filtered_df1 = aggregated_df_year[aggregated_df_year['region'] == selected_region]

    # Calculate the overall min and max values for the y-axes
    min_gdp_capita = aggregated_df_year['gdp_capita'].min()
    max_gdp_capita = aggregated_df_year['gdp_capita'].max()
    
    min_emission = aggregated_df_year['emission'].min()
    max_emission = aggregated_df_year['emission'].max()

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=filtered_df1['year'], y=filtered_df1['gdp_capita'],
                             mode='lines', name='GDP per Capita', yaxis='y'))

    fig.add_trace(go.Scatter(x=filtered_df1['year'], y=filtered_df1['emission'],
                             mode='lines', name='Emissions per Capita', yaxis='y2'))
    
    fig.update_traces(line=dict(width=3))
    
    fig.update_layout(
        title=f'Emissions and GDP per Capita for {selected_region}',
        xaxis_title='',
        yaxis=dict(
            title='GDP per Capita', 
            side='left', 
            showgrid=False,
            range=[min_gdp_capita, max_gdp_capita] 
        ),
        yaxis2=dict(
            title='Emissions per Capita', 
            overlaying='y', 
            side='right', 
            showgrid=False,
            range=[min_emission, max_emission]  
        ),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        legend=dict(
            x=1.1,  
            y=1,    
            traceorder='normal',
            font=dict(
                size=12,
                color='rgb(50, 50, 50)'
            )
        )
    )

    return fig


@app.callback(
    Output('line-plot_region2', 'figure'),
    [Input('region-select', 'value')]
)
def update_line_plot2(selected_region):
    filtered_df2 = aggregated_df_year[aggregated_df_year['region'] == selected_region]

    # Calculate the overall min and max values for the y-axes
    min_life = aggregated_df_year['life_expectancy'].min()
    max_life = aggregated_df_year['life_expectancy'].max()

    min_gdp_capita = aggregated_df_year['gdp_capita'].min()
    max_gdp_capita = aggregated_df_year['gdp_capita'].max()

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=filtered_df2['year'], y=filtered_df2['gdp_capita'],
                             mode='lines', name='GDP per Capita', yaxis='y'))
    
    fig.add_trace(go.Scatter(x=filtered_df2['year'], y=filtered_df2['life_expectancy'],
                             mode='lines', name='Life Expectancy', yaxis='y2'))

    fig.update_traces(line=dict(width=3))
    
    fig.update_layout(
        title=f'GDP per Capita and Life Expectancy total for {selected_region}',
        xaxis_title='',
        yaxis=dict(
            title='GDP per Capita', 
            side='left', 
            showgrid=False,
            range=[min_gdp_capita, max_gdp_capita]  
        ),
        yaxis2=dict(
            title='Life Expectancy', 
            overlaying='y', 
            side='right', 
            showgrid=False,
            range=[min_life, max_life]
        ),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        legend=dict(
            x=1.1, 
            y=1,    
            traceorder='normal',
            font=dict(
                size=12,
                color='rgb(50, 50, 50)'
            )
        )
    )

    return fig

@app.callback(
    Output('line-plot_region3', 'figure'),
    [Input('region-select', 'value')]
)
def update_line_plot3(selected_region):
    # Filter the dataframe for the selected region
    filtered_df3 = aggregated_df_year[aggregated_df_year['region'] == selected_region]
    
    # Calculate the overall min and max values for all sectors
    min_primary_sector = aggregated_df_year['p_agriculture,_hunting,_forestry,_fishing'].min()
    max_primary_sector = aggregated_df_year['p_agriculture,_hunting,_forestry,_fishing'].max()
    
    min_secondary_sector = aggregated_df_year['p_mining_manufacturing_utilities'].min()
    max_secondary_sector = aggregated_df_year['p_mining_manufacturing_utilities'].max()
    
    min_tertiary_sector = aggregated_df_year['p_other_activities'].min()
    max_tertiary_sector = aggregated_df_year['p_other_activities'].max()
    
    # Calculate the combined min and max values for the y-axes
    combined_min = min(min_primary_sector, min_secondary_sector, min_tertiary_sector)
    combined_max = max(max_primary_sector, max_secondary_sector, max_tertiary_sector)

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=filtered_df3['year'], y=filtered_df3['p_agriculture,_hunting,_forestry,_fishing'],
                             mode='lines', name='%Primary Sector in GDP', yaxis='y'))

    fig.add_trace(go.Scatter(x=filtered_df3['year'], y=filtered_df3['p_mining_manufacturing_utilities'],
                             mode='lines', name='%Secondary Sector in GDP', yaxis='y'))

    fig.add_trace(go.Scatter(x=filtered_df3['year'], y=filtered_df3['p_other_activities'],
                             mode='lines', name='%Tertiary Sector in GDP', yaxis='y', line=dict(color='#1f7f73')))
    
    fig.update_traces(line=dict(width=3))
    
    # Update layout with y-axis configurations
    fig.update_layout(
        title=f'Sector Composition of GDP for {selected_region}',
        yaxis=dict(
            title='GDP Sector', 
            side='left', 
            showgrid=False,
            range=[combined_min, combined_max]  
        ),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        xaxis_title='',
        legend=dict(
            x=1.1,
            y=1,
            traceorder='normal',
            font=dict(
                size=12,
                color='rgb(50, 50, 50)'
            )
        )
    )

    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)